In [ ]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import cartopy
import pickle
import xarray as xr
from matplotlib import colors
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import matplotlib as mpl
from scipy.stats import pearsonr

In [ ]:
mpl.__version__

In [ ]:
mpl.rc('font', family='DejaVu Serif') 
# mpl.rc('font', serif='Helvetica Neue') 

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Pr

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_month = pickle.load(pfile)
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-False-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list = pickle.load(pfile)

In [ ]:
solver_list[0].pcs().isel(mode=0).plot()
plt.show()
eof = solver_list[0].eofs().isel(mode=0)
eof = eof.transpose('lat', 'lon')
eof.plot()

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_month_stand = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_stand = pickle.load(pfile)

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-True-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_month_unforced = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-True-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_unforced = pickle.load(pfile)

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-False-month-False-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metrics_mswep = pickle.load(pfile)
mswep_pc = metrics_mswep['obs_pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-False-month-False-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metrics_gpcp = pickle.load(pfile)
gpcp_pc = metrics_gpcp['obs_pc']
pc1 = metrics_gpcp['pc']
pc2 = metrics_mswep['pc']

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/pr-metrics-stand-False-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metrics_mswep_month = pickle.load(pfile)
mswep_pc_month = metrics_mswep_month['obs_pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-False-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metrics_gpcp_month = pickle.load(pfile)
gpcp_pc_month = metrics_gpcp_month['obs_pc']
pc1_month = metrics_gpcp_month['pc']
pc2_month = metrics_mswep_month['pc']

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-True-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metrics_gpcp_month_stand = pickle.load(pfile)
pc1_month_stand = metrics_gpcp_month_stand['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-True-month-False-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metrics_gpcp_stand = pickle.load(pfile)
pc1_stand = metrics_gpcp_stand['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-True-month-False-unforced-True-joint-False-REGEN-mask', 'rb') as pfile:
    metrics_gpcp_unforced = pickle.load(pfile)
pc1_unforced = metrics_gpcp_unforced['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/GPCP/pr-metrics-stand-True-month-True-unforced-True-joint-False-REGEN-mask', 'rb') as pfile:
    metrics_gpcp_month_unforced = pickle.load(pfile)
pc1_month_unforced = metrics_gpcp_month_unforced['pc']

In [ ]:
pc1.time.data[0]

In [ ]:
pc1_month.time.data[0]
pc1_time = pc1_month.indexes['time'].to_datetimeindex()

In [ ]:
def plot_eofs(solver_list_month, solver_list, column, fig, gs, norm, cmap='BrBG', left=True, skip=False):
    eof_all = None
    for i in range(6):
        eof = solver_list_month[i].eofs().isel(mode=0).transpose('lat', 'lon')
        variance = str(solver_list_month[i].varianceFraction().data[0].round(2))
        ax = fig.add_subplot(gs[i, column], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        ax.set_title(months[i]+' '+variance, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        if eof_all is None:
            eof_all = eof.data
        else:
            eof_all+=eof.data
    for i in range(6, 12):
        eof = solver_list_month[i].eofs().isel(mode=0).transpose('lat', 'lon')
        variance = str(solver_list_month[i].varianceFraction().data[0].round(2))
        if skip:
            ax = fig.add_subplot(gs[11-i, column+2], projection=cartopy.crs.Robinson(central_longitude=180))
        else:
            ax = fig.add_subplot(gs[11-i, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        ax.set_title(months[i]+' '+variance, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        if eof_all is None:
            eof_all = eof.data
        else:
            eof_all+=eof.data
    '''if left:
        ax = fig.add_subplot(gs[6, column], projection=cartopy.crs.Robinson(central_longitude=180))
    else:
        ax = fig.add_subplot(gs[6, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.contourf(eof.lon, eof.lat, eof_all/12, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
    ax.set_title('Mean EOF')
    if left:
        ax = fig.add_subplot(gs[7, column], projection=cartopy.crs.Robinson(central_longitude=180))
    else:
        ax = fig.add_subplot(gs[7, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
    
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.set_title('AllM '+variance)'''
    eof = solver_list[0].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list[0].varianceFraction().data[0].round(2))
    return eof_all/12, eof, variance

In [ ]:
sns_colors = sns.color_palette('muted', 3)
sns_colors

In [ ]:
pc1.plot()
pc1_stand.plot()

In [ ]:
def plot_eofs_multicols(solver_list_month, solver_list, column, fig, gs, norm, cmap='BrBG', left=True, skip=False):
    eof_all = None
    for i in range(6):
        eof = solver_list_month[i].eofs().isel(mode=0).transpose('lat', 'lon')
        variance = str(solver_list_month[i].varianceFraction().data[0].round(2))
        ax = fig.add_subplot(gs[i, column:column+3], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        ax.set_title(months[i]+' '+variance, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        if eof_all is None:
            eof_all = eof.data
        else:
            eof_all+=eof.data
    for i in range(6, 12):
        eof = solver_list_month[i].eofs().isel(mode=0).transpose('lat', 'lon')
        variance = str(solver_list_month[i].varianceFraction().data[0].round(2))
        if skip:
            ax = fig.add_subplot(gs[11-i, column+3:column+6], projection=cartopy.crs.Robinson(central_longitude=180))
        else:
            ax = fig.add_subplot(gs[11-i, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
        ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        ax.set_title(months[i]+' '+variance, 
                    fontsize=12)
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        if eof_all is None:
            eof_all = eof.data
        else:
            eof_all+=eof.data
    '''if left:
        ax = fig.add_subplot(gs[6, column], projection=cartopy.crs.Robinson(central_longitude=180))
    else:
        ax = fig.add_subplot(gs[6, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.contourf(eof.lon, eof.lat, eof_all/12, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
    ax.set_title('Mean EOF')
    if left:
        ax = fig.add_subplot(gs[7, column], projection=cartopy.crs.Robinson(central_longitude=180))
    else:
        ax = fig.add_subplot(gs[7, column+1], projection=cartopy.crs.Robinson(central_longitude=180))
    
    ax.contourf(eof.lon, eof.lat, eof, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
    ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
    ax.set_title('AllM '+variance)'''
    eof = solver_list[0].eofs().isel(mode=0).transpose('lat', 'lon')
    variance = str(solver_list[0].varianceFraction().data[0].round(2))
    return eof_all/12, eof, variance

In [ ]:
from datetime import datetime

len(pc1_time)
label = [datetime.strftime(pc1_time[0], '%Y-%m'), datetime.strftime(pc1_time[100], '%Y-%m'),
         datetime.strftime(pc1_time[200], '%Y-%m'), datetime.strftime(pc1_time[300], '%Y-%m'),
         datetime.strftime(pc1_time[400], '%Y-%m')]

In [ ]:
vmin = -0.1
vmax = 0.1
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(8, 13, figure=fig, width_ratios=[1, 1, 1,  1, 1, 1,  0.25, 1, 1, 1, 1, 1,1], wspace=0)
ind = 0
mean_eof1, eof_all1, variance1 = plot_eofs_multicols(column=0, solver_list_month=solver_list_month, 
                                                     solver_list=solver_list, norm=norm, 
                                                     gs=gs, fig=fig, left=True, skip=True)
mean_eof2, eof_all2, variance2 = plot_eofs_multicols(column=7, solver_list_month=solver_list_month_stand, 
                                                     solver_list=solver_list_stand, norm=norm, 
                                                     gs=gs, fig=fig, left=False, skip=True)
'''mean_eof3, eof_all3, variance3 = plot_eofs(column=6, solver_list_month=solver_list_month_unforced, solver_list=solver_list_unforced, 
                                           norm=norm, gs=gs, fig=fig)'''

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
ax = fig.add_subplot(gs[6:8, 4:9])
ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='-.')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='-.')
ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2])
ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], linestyle='-.')
ax.set_xticks([pc1_time[0], pc1_time[100], pc1_time[200], pc1_time[300], pc1_time[400]])
ax.set_xticklabels(label)
plt.legend(ncol=3, bbox_to_anchor=(1.5, -.1))
fig.text(x=0.21, y=1., s='Anomaly', fontsize=14, weight='bold')
fig.text(x=0.66, y=1., s='StdAnomaly', fontsize=14, weight='bold')

ax = fig.add_subplot(gs[6, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof1, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[6, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof2, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[7, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, -eof_all1, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm)
ax.set_title('AllM '+variance1)
ax = fig.add_subplot(gs[7, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all2, transform=cartopy.crs.PlateCarree(), cmap='BrBG', norm=norm)
ax.set_title('AllM '+variance2)
# fig.text(x=0.78, y=1., s='UnforcedAnomaly', fontsize=14, weight='bold')
cbar_ax = fig.add_axes([-0.01, 0.05, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap='BrBG'), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
plt.savefig('pr-land-EOF-pc.png', dpi=180, bbox_inches='tight')

# Monmaxpr

In [ ]:
variable = 'monmaxpr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_month = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-False-month-False-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list = pickle.load(pfile)

with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_month_stand = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_stand = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-True-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_month_unforced = pickle.load(pfile)
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-False-unforced-True-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_unforced = pickle.load(pfile)

In [ ]:
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-False-month-False-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1 = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-False-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_month = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-True-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_month_stand = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-True-month-False-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_stand = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-True-month-False-unforced-True-joint-False-REGEN-mask', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_unforced = metric['pc']
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/1979_2022/MSWEP/monmaxpr-metrics-stand-True-month-True-unforced-True-joint-False-REGEN-mask', 'rb') as pfile:
    metric = pickle.load(pfile)
pc1_month_unforced = metric['pc']

In [ ]:
pc1_month.time.data[0]
pc1_time = pc1_month.indexes['time'].to_datetimeindex()
sns_colors = sns.color_palette('muted', 3)
sns_colors

In [ ]:
solver_list_unforced[0].eofs().isel(mode=0).plot()

In [ ]:
from datetime import datetime

len(pc1_time)
label = [datetime.strftime(pc1_time[0], '%Y-%m'), datetime.strftime(pc1_time[100], '%Y-%m'),
         datetime.strftime(pc1_time[200], '%Y-%m'), datetime.strftime(pc1_time[300], '%Y-%m'),
         datetime.strftime(pc1_time[400], '%Y-%m')]

In [ ]:
vmin = -0.09
vmax = 0.09
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
cmap = 'BrBG'
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(8, 13, figure=fig, width_ratios=[1, 1, 1,  1, 1, 1,  0.25, 1, 1, 1, 1, 1,1], wspace=0)
ind = 0
mean_eof1, eof_all1, variance1 = plot_eofs_multicols(column=0, solver_list_month=solver_list_month, 
                                                     solver_list=solver_list, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=True, skip=True)
mean_eof2, eof_all2, variance2 = plot_eofs_multicols(column=7, solver_list_month=solver_list_month_stand, 
                                                     solver_list=solver_list_stand, norm=norm, cmap=cmap,
                                                     gs=gs, fig=fig, left=False, skip=True)

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
ax = fig.add_subplot(gs[6:8, 4:9])
ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='-.')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='-.')
ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2])
ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], linestyle='-.')
ax.set_xticks([pc1_time[0], pc1_time[100], pc1_time[200], pc1_time[300], pc1_time[400]])
ax.set_xticklabels(label)
plt.legend(ncol=3, bbox_to_anchor=(1.5, -.1))
fig.text(x=0.21, y=1., s='Anomaly', fontsize=14, weight='bold')
fig.text(x=0.66, y=1., s='StdAnomaly', fontsize=14, weight='bold')

ax = fig.add_subplot(gs[6, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[6, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, mean_eof2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
ax.set_title('Mean EOF')
ax = fig.add_subplot(gs[7, 0:3], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all1, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
ax.set_title('AllM '+variance1)
ax = fig.add_subplot(gs[7, 10:13], projection=cartopy.crs.Robinson(central_longitude=180))
ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
ax.contourf(eof_all1.lon, eof_all1.lat, eof_all2, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
ax.set_title('AllM '+variance2)
cbar_ax = fig.add_axes([-0.01, 0.05, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
plt.savefig('monmaxpr-land-EOF-pc.png', dpi=180, bbox_inches='tight')

In [ ]:
vmin = -0.055
vmax = 0.055
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig = plt.figure(figsize=(14, 14))
gs = GridSpec(9, 8, figure=fig, width_ratios=[1, 1, 0.2, 1, 1, 0.2, 1, 1])
ind = 0
plot_eofs(column=0, solver_list_month=solver_list_month, solver_list=solver_list, norm=norm, gs=gs, fig=fig, cmap='BrBG')
plot_eofs(column=3, solver_list_month=solver_list_month_stand, solver_list=solver_list_stand, norm=norm, gs=gs, fig=fig, cmap='BrBG')
plot_eofs(column=6, solver_list_month=solver_list_month_unforced, solver_list=solver_list_unforced, norm=norm, gs=gs, fig=fig, cmap='BrBG')

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.2)
ax = fig.add_subplot(gs[8, 2:6])
ax.plot(pc1_time, pc1, label='AllM Anomaly', color=sns_colors[0])
ax.plot(pc1_time, pc1_month, label='MbyM Anomaly', color=sns_colors[0], linestyle='-.')
ax.plot(pc1_time, pc1_stand, label='AllM StdAnomaly', color=sns_colors[1])
ax.plot(pc1_time, pc1_month_stand, label='MbyM StdAnomaly', color=sns_colors[1], linestyle='-.')
ax.plot(pc1_time, pc1_unforced, label='AllM UnforcedAnomaly', color=sns_colors[2])
ax.plot(pc1_time, pc1_month_unforced, label='MbyM UnforcedAnomaly', color=sns_colors[2], linestyle='-.')
plt.legend(ncol=3, bbox_to_anchor=(1.05, -.2))
fig.text(x=0.13, y=1., s='Anomaly', fontsize=14, weight='bold')
fig.text(x=0.46, y=1., s='StdAnomaly', fontsize=14, weight='bold')
fig.text(x=0.78, y=1., s='UnforcedAnomaly', fontsize=14, weight='bold')
cbar_ax = fig.add_axes([0.04, 0.15, 0.01, 0.15])
cb = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm, cmap='BrBG'), cax=cbar_ax, extend='both')
cbar_ax.yaxis.set_ticks_position('left')
# plt.savefig('monmaxpr-EOF-pc.png', dpi=180, bbox_inches='tight')